# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [132]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [133]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0124"

## Built-in LangChain tools

In [134]:
#!pip install -U wikipedia 
#!pip install -U numexpr

In [135]:
from langchain_experimental.agents.agent_toolkits.python.base import create_python_agent
from langchain.agents import create_react_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.tools.python.tool import PythonREPL
from langchain_openai import ChatOpenAI

In [136]:
llm_model = "gpt-3.5-turbo"

In [137]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [138]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [139]:
from langchain import hub
from langchain_community.llms import OpenAI
from langchain.agents import AgentExecutor, create_react_agent


In [140]:
prompt = hub.pull("hwchase17/react") #  , api_url="https://api.hub.langchain.com")

In [141]:
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


In [142]:
agent = create_react_agent(tools=tools,llm=llm, prompt=prompt)

In [143]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [144]:
# import RunnableConfig
from langchain_core.runnables import RunnableConfig

In [145]:
agent_executor.verbose = True

In [146]:
agent_executor.invoke({"input":"how much is 25% of 370"}, config=RunnableConfig(allow_repl=True))



> Entering new AgentExecutor chain...
I need to calculate 25% of 370.
Action: Calculator
Action Input: 0.25 * 370Answer: 92.5I now know the final answer
Final Answer: 25% of 370 is 92.5.

> Finished chain.


{'input': 'how much is 25% of 370', 'output': '25% of 370 is 92.5.'}

## Wikipedia example

In [91]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"


In [92]:
agent_executor.invoke({"input":question}, config=RunnableConfig(allow_repl=True))



> Entering new AgentExecutor chain...
I need to find out what book Tom M. Mitchell wrote. I can use Wikipedia to find this information.
Action: Wikipedia
Action Input: Tom M. MitchellPage: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the Scho

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Tom M. Mitchell is the author of the textbook "Machine Learning".'}

## Python Agent

In [93]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [94]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [95]:
agent.invoke(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I need to sort the list of customers first by their last name and then by their first name. I can use the built-in sorted function in Python for this. The sorted function takes an iterable and returns a new sorted list of the elements in the iterable. I can pass a function to the key parameter of the sorted function to specify a custom sort order. In this case, I want to sort by the last name and then the first name, so I can pass a function that returns a tuple of the last name and first name. The sorted function will then sort the customers based on this tuple. Action: Python_REPL
Action Input: 
```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)
```
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly

{'input': "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': "[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"}

#### View detailed outputs of the chains

In [ ]:
import langchain
langchain.debug=True
agent.invoke(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

## Define your own tool

In [96]:
#!pip install DateTime

In [98]:
from langchain.tools import tool
from datetime import date

In [99]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathematics should occur \
    outside this function."""
    return str(date.today())

In [102]:
agent = create_react_agent(tools=tools + [time],llm=llm, prompt=prompt)

In [103]:
agent_executor = AgentExecutor(agent=agent, tools=tools + [time])

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [106]:
try:
    result = agent_executor.invoke({"input":"what is todays date?"}, config=RunnableConfig(allow_repl=True))
except: 
    print("exception on external access")

In [107]:
result

{'input': 'what is todays date?', 'output': "Today's date is March 11, 2024."}

Reminder: Download your notebook to you local computer to save your work.